In [222]:
import pandas as pd
import numpy as np
import re

In [223]:
jobsdata = pd.read_csv("jobs.csv")

In [224]:
jobsdata.tail(5)
#salary parsing
#Company Name text only
#Location(Only get state)
#Age of company
#Parse JD to pick up some key words.

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
951,951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"See Tickets, TicketWeb, Vendini"
952,952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,-1
953,953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),-1
954,954,Data Engineer,-1,Loading...\n\nTitle: Data Engineer\n\nLocation...,4.8,IGNW\n4.8,"Austin, TX","Portland, OR",201 to 500 employees,2015,Company - Private,IT Services,Information Technology,$25 to $50 million (USD),Slalom
955,955,Research Scientist – Security and Privacy,$61K-$126K (Glassdoor est.),Returning Candidate? Log back in to the Career...,3.6,Riverside Research Institute\n3.6,"Beavercreek, OH","Arlington, VA",501 to 1000 employees,1967,Nonprofit Organization,Federal Agencies,Government,$50 to $100 million (USD),-1


In [225]:
#Removing rows where Salary Estimate is not equal to -1.
jobsdata = jobsdata[jobsdata['Salary Estimate'] != '-1']

In [226]:
jobsdata['hourly'] = jobsdata['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0) #making a boolean column for per hour salary
jobsdata['employer_provided'] = jobsdata['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0) #making a boolean column for employed provided salary

In [227]:
salary=jobsdata['Salary Estimate'].apply(lambda x: x.split('(')[0])
salary
minus_kd = salary.apply(lambda x: x.replace('$','').replace('K','')) #removing K and $
minus_hr = minus_kd.apply(lambda x: x.lower().replace('per hour','')) #removing per hour keyword
minus_employer_provided = minus_hr.apply(lambda x: x.lower().replace('employer provided salary:','')) #removing employer provided keyword
jobsdata['min salary'] = minus_employer_provided.apply(lambda x: int(x.split('-')[0])) #getting min from salary 
jobsdata['max salary'] = minus_employer_provided.apply(lambda x: int(x.split('-')[1])) #getting min from salary

In [228]:
jobsdata['Average Salary'] = (jobsdata['min salary']+jobsdata['max salary'])/2 # creating field for avg salary. Salary colum is done

In [229]:
#To get only text in company name, we get company name where rating is not available, for the cases where rating is available we remove the last 3 characters from company name
jobsdata['Company_text'] = jobsdata.apply(lambda x: x['Company Name'] if x['Rating']<0 else x['Company Name'][:-4], axis=1)

In [230]:
#to get state from the location column and to make a boolean column to check if job location is same as headquaters
jobsdata['State'] = jobsdata['Location'].apply(lambda x: x.split(',')[1])
jobsdata['Same_state'] = jobsdata.apply(lambda x: 1 if x['Location']==x['Headquarters'] else 0, axis=1)

In [231]:
#to get age of company
jobsdata['Age'] = jobsdata['Founded'].apply(lambda x: x if x<1 else 2020-x)

In [232]:
#to check for specific keywords in JD
jobsdata['python'] = jobsdata['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0 )
jobsdata['spark'] = jobsdata['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0 )
jobsdata['R'] = jobsdata['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() else 0 )
jobsdata['aws'] = jobsdata['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0 )
jobsdata['excel'] = jobsdata['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0 )

In [233]:
jobsdata = jobsdata.drop(['Unnamed: 0'], axis=1)

In [236]:
jobsdata.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min salary', 'max salary',
       'Average Salary', 'Company_text', 'State', 'Same_state', 'Age',
       'python', 'spark', 'R', 'aws', 'excel'],
      dtype='object')

In [245]:
jobsdata.to_csv('data_cleaned.csv',index=0)